---
title: "A look at different scenarios - Gains and Losses"
format:
  html:
    code-fold: true
    code-summary: "Show the code"
  docx: 
    fig-height: 11.6 in
---

In [1]:
import pandas as pd
from fagrants_module import fagrants_gccsa as fagrants

def weighted_group_delta(grouped_df):
    return_dict = {}

    for year in range(2025, 2029):
        total_weight = grouped_df[f"ERP_{year}"].mean()

        return_dict[f"Population_{year}"] = total_weight

        weighted_mean = (
            grouped_df[f"Grant Delta_{year}"] * grouped_df[f"ERP_{year}"]
        ).sum() / grouped_df[f"ERP_{year}"].sum()

        return_dict[f"Grant Delta_{year}"] = weighted_mean

    return pd.Series(return_dict)

def grants_simulation(data, pc_min, pc_min_base):
    inst = fagrants.fagrants_model(
    data_path=data,
    per_capita_minimum=pc_min,
    per_capita_minimum_base=pc_min_base,
    )

    sim_grants = inst.run()

    sim_grants["Sim_i"] = sim_grants["UID"].str.split("_").str[1].astype(int)

    for year in range(2025, 2029):
        sim_grants[f"Grant Delta_{year}"] = (
            sim_grants[f"Grant_{year}"] - sim_grants[f"Grant_base_{year}"]
        )

    return sim_grants

def grants_delta(sim_grants):
    sim_grants_grouped = (
        sim_grants.groupby(["Jurisdiction", "ACLG", "LGA"])
        .apply(weighted_group_delta, include_groups=False)
        .reset_index()
    )

    pop_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "LGA"],
        value_vars=[f"Population_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Population",
    )

    sim_grants_melted = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction","ACLG", "LGA"],
        value_vars=[f"Grant Delta_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG ($)",
    )

    for year in range(2025, 2029):
        sim_grants_grouped[f"Grant Delta Per Capita_{year}"] = sim_grants_grouped[f"Grant Delta_{year}"]/ sim_grants_grouped[f"Population_{year}"]

    sim_grants_melted_per_capita = pd.melt(
        sim_grants_grouped,
        id_vars=["Jurisdiction", "ACLG", "LGA"],
        value_vars=[f"Grant Delta Per Capita_{year}" for year in range(2025, 2029)],
        var_name="Year",
        value_name="Expected Change in GPG Per Capita ($)",
    )

    sim_grants_melted["Year"] = sim_grants_melted["Year"].str.split("_").str[1].astype(int)

    sim_grants_melted["Expected Change in GPG Per Capita ($)"] = sim_grants_melted_per_capita["Expected Change in GPG Per Capita ($)"]

    sim_grants_melted["Population"] = pop_melted["Population"]

    return sim_grants_melted

In [2]:
sim_grants_30 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.30, 0.30)

sim_grants_20 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.20, 0.30)

sim_grants_10 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.10, 0.30)

sim_grants_0 = grants_simulation("./Data/FA Grants Tables - Python.xlsx",
    0.00, 0.30)

In [3]:
sim_grants_30_melted = grants_delta(sim_grants_30)

sim_grants_20_melted = grants_delta(sim_grants_20)

sim_grants_10_melted = grants_delta(sim_grants_10)

sim_grants_0_melted = grants_delta(sim_grants_0)

In [4]:
from datetime import datetime as tm

with pd.ExcelWriter(f"./Data/Output/Simulation Results {tm.today().strftime("%d_%m_%Y")}.xlsx") as writer:
    sim_grants_30_melted.to_excel(writer, sheet_name = "30% Minimum Grant")
    sim_grants_20_melted.to_excel(writer, sheet_name = "20% Minimum Grant")
    sim_grants_10_melted.to_excel(writer, sheet_name = "10% Minimum Grant")
    sim_grants_0_melted.to_excel(writer, sheet_name = "0% Minimum Grant")

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.precision', 1)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

def display_data(data, state):
    sub = data[data["Jurisdiction"] == state]

    k = sub.groupby(["Jurisdiction", "ACLG", "Year"]).agg(
        {
            "Expected Change in GPG ($)": lambda x: sum(x * sub.loc[x.index, "Population"])/sub.loc[x.index, "Population"].sum(),
            "Expected Change in GPG Per Capita ($)": lambda x: sum(x * sub.loc[x.index, "Population"])/sub.loc[x.index, "Population"].sum(),
            "Population": 'sum'
        }
    )

    display(k)

## A 20% Per Capita Minimum Grant

### New South Wales

In [6]:
display_data(sim_grants_20_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                    559076.9   
                     2026                    579720.6   
                     2027                    601249.1   
                     2028                    622921.2   
                RAM  2025                    524328.1   
                     2026                    544384.3   
                     2027                    563723.0   
                     2028                    584608.3   
                RAS  2025                    726283.7   
                     2026                    750319.7   
                     2027                    776589.1   
                     2028                    802971.4   
                RAV  2025                    563366.1   
                     2026                    585349.6   
                     2027                    607296.5   
                     2028                    629193.1   
                RTL  2025                    833684.4   
                     2026                    863591.3   
                     2027                    897560.2   
                     2028                    932487.2   
                RTM  2025                    918064.5   
                     2026                    954571.0   
                     2027                    992098.6   
                     2028                   1034597.2   
                RTX  2025                     38584.2   
                     2026                     40141.4   
                     2027                     41390.5   
                     2028                     42932.7   
                UCC  2025                  -2082509.3   
                     2026                  -2164906.0   
                     2027                  -2248760.1   
                     2028                  -2334804.7   
                UDL  2025                   -711911.5   
                     2026                   -740081.1   
                     2027                   -768746.2   
                     2028                   -798161.6   
                UDM  2025                   -393796.1   
                     2026                   -409403.6   
                     2027                   -425297.7   
                     2028                   -441605.7   
                UDS  2025                   -113922.8   
                     2026                   -118407.3   
                     2027                   -123062.6   
                     2028                   -127866.8   
                UDV  2025                  -2217559.3   
                     2026                  -2305549.0   
                     2027                  -2395238.1   
                     2028                  -2486664.1   
                UFL  2025                   -642324.7   
                     2026                   -666912.6   
                     2027                   -694634.5   
                     2028                   -724629.3   
                UFM  2025                   -372818.1   
                     2026                   -387724.6   
                     2027                   -403591.5   
                     2028                   -418845.6   
                UFV  2025                  -1204399.4   
                     2026                  -1254341.5   
                     2027                  -1304067.4   
                     2028                  -1361857.1   
                URL  2025                    150902.0   
                     2026                    159369.6   
                     2027                    169182.2   
                     2028                    181127.7   
                URM  2025                    274040.0   
                     2026                    285473.4   
                     2027                    298188.9   
                     2028                    311724.7   
                URS  2025            

### Victoria

In [7]:
display_data(sim_grants_20_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                    423667.5   
                  2026                    441726.0   
                  2027                    461579.2   
                  2028                    479924.7   
             RAM  2025                    471442.2   
                  2026                    492123.3   
                  2027                    513264.5   
                  2028                    533484.9   
             RAV  2025                    407315.4   
                  2026                    426133.8   
                  2027                    444407.3   
                  2028                    463010.1   
             UCC  2025                  -1622527.7   
                  2026                  -1693812.6   
                  2027                  -1765786.3   
                  2028                  -1839477.0   
             UDL  2025                   -838278.7   
                  2026                   -875106.5   
                  2027                   -912074.3   
                  2028                   -950213.1   
             UDV  2025                  -1003166.3   
                  2026                  -1047777.4   
                  2027                  -1093008.5   
                  2028                  -1139890.2   
             UFL  2025                    289589.0   
                  2026                    304689.3   
                  2027                    317503.9   
                  2028                    331308.9   
             UFM  2025                     -4951.8   
                  2026                     -6404.6   
                  2027                     -7709.4   
                  2028                     -8073.5   
             UFS  2025                    313808.6   
                  2026                    327019.9   
                  2027                    338556.7   
                  2028                    354465.9   
             UFV  2025                   -133459.8   
                  2026                   -147472.0   
                  2027                   -160571.4   
                  2028                   -170099.7   
             URL  2025                    835413.4   
                  2026                    877986.3   
                  2027                    921566.0   
                  2028                    958594.3   
             URM  2025                    683562.2   
                  2026                    715788.9   
                  2027                    747671.2   
                  2028                    780073.7   
             URS  2025                    577404.5   
                  2026                    601730.0   
                  2027                    626216.8   
                  2028                    654836.1   
             URV  2025                    307837.1   
                  2026                    328450.7   
                  2027                    352654.9   
                  2028                    375051.3   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Victoria     RAL  2025                                   60.3     50462.0  
                  2026                                   62.4     50851.0  
                  2027                                   64.7     51217.0  
                  2028                                   66.8     51602.0  
             RAM  2025                                  120.4      3917.0  
                  2026                                  124.7      3947.0  
                  2027                                  129.1      3976.0  
                  2028                                  133.2      4005.0  
             RAV  2025                                   26.3    246987.0  
                  2026                                 

### Queensland

In [8]:
display_data(sim_grants_20_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                   1346052.2   
                  2026                   1408880.3   
                  2027                   1469605.5   
                  2028                   1528112.7   
             RAV  2025                    832842.6   
                  2026                    868570.1   
                  2027                    902070.9   
                  2028                    938777.2   
             RTL  2025                   1163319.6   
                  2026                   1207426.1   
                  2027                   1253443.3   
                  2028                   1297850.2   
             RTM  2025                    853855.4   
                  2026                    889334.6   
                  2027                    924137.3   
                  2028                    960370.7   
             RTS  2025                    841600.5   
                  2026                    876452.5   
                  2027                    912245.7   
                  2028                    946943.9   
             RTX  2025                    833744.2   
                  2026                    868923.5   
                  2027                    902894.5   
                  2028                    937988.0   
             UCC  2025                 -12019933.1   
                  2026                 -12536627.3   
                  2027                 -13047239.9   
                  2028                 -13569107.2   
             UFM  2025                   -352763.9   
                  2026                   -367449.7   
                  2027                   -381939.2   
                  2028                   -396657.6   
             UFS  2025                   -160477.7   
                  2026                   -167693.5   
                  2027                   -174794.2   
                  2028                   -182181.7   
             URL  2025                   -988898.6   
                  2026                  -1027295.6   
                  2027                  -1065786.1   
                  2028                  -1104964.1   
             URM  2025                   -260592.6   
                  2026                   -270046.2   
                  2027                   -279822.8   
                  2028                   -289763.1   
             URS  2025                    256070.2   
                  2026                    267683.8   
                  2027                    277385.2   
                  2028                    288576.8   
             URV  2025                  -3649633.3   
                  2026                  -3797448.6   
                  2027                  -3944761.0   
                  2028                  -4094976.7   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Queensland   RAM  2025                                  299.7      4492.0  
                  2026                                  309.8      4548.0  
                  2027                                  319.4      4601.0  
                  2028                                  328.3      4654.0  
             RAV  2025                                   62.5    105395.0  
                  2026                                   64.4    106725.0  
                  2027                                   66.1    107956.0  
                  2028                                   68.0    109216.0  
             RTL  2025                                  220.1     47507.0  
                  2026                                  225.7     48105.0  
                  2027                                  231.8     48660.0  
                  2028                                  237.5     49227.0  
             RTM  2025                     

### South Australia

In [9]:
display_data(sim_grants_20_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                    212684.4   
                     2026                    219892.2   
                     2027                    227191.5   
                     2028                    234722.9   
                RAM  2025                    163044.6   
                     2026                    168226.4   
                     2027                    173636.9   
                     2028                    179607.0   
                RAS  2025                    134253.3   
                     2026                    138628.3   
                     2027                    143040.0   
                     2028                    147752.8   
                RAV  2025                    289941.3   
                     2026                    300884.7   
                     2027                    311813.0   
                     2028                    322678.1   
                RSG  2025                    -38977.3   
                     2026                    -39938.5   
                     2027                    -40899.0   
                     2028                    -41872.5   
                UCC  2025                   -250849.5   
                     2026                   -259447.9   
                     2027                   -268100.8   
                     2028                   -277072.5   
                UDL  2025                   -974824.0   
                     2026                  -1008384.5   
                     2027                  -1042086.3   
                     2028                  -1076906.0   
                UDM  2025                   -482957.1   
                     2026                   -499509.4   
                     2027                   -516178.3   
                     2028                   -533442.8   
                UDS  2025                   -172712.9   
                     2026                   -178634.9   
                     2027                   -184595.9   
                     2028                   -190770.8   
                UDV  2025                   -799165.1   
                     2026                   -826818.2   
                     2027                   -857654.0   
                     2028                   -884642.5   
                UFL  2025                    963023.3   
                     2026                    995055.3   
                     2027                   1027754.5   
                     2028                   1051904.5   
                UFM  2025                   -376187.4   
                     2026                   -389078.5   
                     2027                   -402069.2   
                     2028                   -415515.3   
                UFS  2025                     49476.7   
                     2026                     51839.6   
                     2027                     54726.6   
                     2028                     58488.7   
                UFV  2025                   -911207.1   
                     2026                   -942165.6   
                     2027                   -973515.4   
                     2028                  -1007750.7   
                URM  2025                   -235744.8   
                     2026                   -244517.6   
                     2027                   -252908.3   
                     2028                   -261958.1   
                URS  2025                    274188.3   
                     2026                    283981.3   
                     2027                    293916.8   
                     2028                    303871.6   

                           Expected Change in GPG Per Capita ($)  Population  
Jurisdiction    ACLG Year                                                     
South Australia RAL  2025                                   25.1     83192.0  
                     2026  

### Western Australia

In [10]:
display_data(sim_grants_20_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                    200594.5   
                       2026                    209850.1   
                       2027                    218958.7   
                       2028                    228633.3   
                  RAM  2025                    110258.8   
                       2026                    115457.8   
                       2027                    120621.7   
                       2028                    125496.9   
                  RAS  2025                    155043.0   
                       2026                    161784.0   
                       2027                    168496.2   
                       2028                    175253.3   
                  RAV  2025                    203523.0   
                       2026                    212204.3   
                       2027                    220799.6   
                       2028                    230604.4   
                  RSG  2025                    -91290.7   
                       2026                    -93883.9   
                       2027                    -95685.6   
                       2028                    -98206.5   
                  RTL  2025                    206562.3   
                       2026                    216559.7   
                       2027                    226592.8   
                       2028                    235825.2   
                  RTM  2025                    273358.9   
                       2026                    285838.2   
                       2027                    298587.7   
                       2028                    310147.3   
                  RTS  2025                    240975.2   
                       2026                    251851.2   
                       2027                    261825.4   
                       2028                    272540.0   
                  RTX  2025                    366748.5   
                       2026                    382274.6   
                       2027                    397236.9   
                       2028                    413220.9   
                  UCC  2025                   -299424.0   
                       2026                   -313151.1   
                       2027                   -326483.4   
                       2028                   -340066.3   
                  UDL  2025                  -1056293.1   
                       2026                  -1104736.8   
                       2027                  -1151759.5   
                       2028                  -1199674.0   
                  UDM  2025                   -476927.9   
                       2026                   -498798.0   
                       2027                   -520030.9   
                       2028                   -541663.0   
                  UDS  2025                   -222418.4   
                       2026                   -232622.8   
                       2027                   -242521.2   
                       2028                   -252608.7   
                  UDV  2025                  -1698224.7   
                       2026                  -1776429.0   
                       2027                  -1852435.2   
                       2028                  -1929544.2   
                  UFL  2025                   -904679.2   
                       2026                   -946165.5   
                       2027                   -986436.7   
                       2028                  -1027470.7   
                  UFM  2025                   -424079.1   
                       2026                   -444241.8   
                       2027                   -463085.4   
                       2028                   -483545.6   
                  UFS  2025                    -80928.4   
                       2026                    -85409.6   
                 

### Tasmania

In [11]:
display_data(sim_grants_20_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                    150325.5   
                  2026                    154471.9   
                  2027                    159323.0   
                  2028                    163650.4   
             RAM  2025                     63708.7   
                  2026                     65192.6   
                  2027                     66851.6   
                  2028                     69195.5   
             RAS  2025                    128275.8   
                  2026                    131151.8   
                  2027                    134547.1   
                  2028                    138721.0   
             RAV  2025                     79767.6   
                  2026                     82523.0   
                  2027                     84851.1   
                  2028                     87791.3   
             UCC  2025                   -591722.8   
                  2026                   -608956.7   
                  2027                   -626868.2   
                  2028                   -647099.3   
             UFM  2025                   -546946.2   
                  2026                   -562844.4   
                  2027                   -579350.5   
                  2028                   -598003.7   
             UFS  2025                     44871.7   
                  2026                     46340.1   
                  2027                     47338.7   
                  2028                     47718.8   
             URM  2025                   -501049.9   
                  2026                   -514537.9   
                  2027                   -527564.9   
                  2028                   -542609.4   
             URS  2025                    115571.7   
                  2026                    119021.0   
                  2027                    121843.9   
                  2028                    126000.8   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Tasmania     RAL  2025                                   22.3     41480.0  
                  2026                                   22.9     41569.0  
                  2027                                   23.6     41646.0  
                  2028                                   24.2     41736.0  
             RAM  2025                                   13.6     23683.0  
                  2026                                   13.8     23735.0  
                  2027                                   14.2     23779.0  
                  2028                                   14.6     23831.0  
             RAS  2025                                   98.0      2672.0  
                  2026                                  100.0      2678.0  
                  2027                                  102.4      2683.0  
                  2028                                  105.4      2688.0  
             RAV  2025                                    6.0    105349.0  
                  2026                                    6.2    105628.0  
                  2027                                    6.4    105922.0  
                  2028                                    6.6    106243.0  
             UCC  2025                                   -8.7     68013.0  
                  2026                                   -8.9     68307.0  
                  2027                                   -9.1     68707.0  
                  2028                                   -9.4     69107.0  
             UFM  2025                                   -8.7    173305.0  
                  2026                                   -8.9    173977.0  
                  2027                                   -9.1    174856.0  
                  2028                                   -9.4    175746

### Northern Territory

In [12]:
display_data(sim_grants_20_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                    253805.1   
                        2026                    261925.7   
                        2027                    270491.6   
                        2028                    279959.8   
                   RTM  2025                     18882.4   
                        2026                     19833.5   
                        2027                     20479.5   
                        2028                     21516.9   
                   RTS  2025                     -4393.9   
                        2026                     -4509.1   
                        2027                     -4614.4   
                        2028                     -4734.1   
                   RTX  2025                      2357.7   
                        2026                      2454.8   
                        2027                      2537.8   
                        2028                      2640.7   
                   UCC  2025                   -760500.0   
                        2026                   -787022.9   
                        2027                   -814704.0   
                        2028                   -846407.0   
                   UFS  2025                   -208357.5   
                        2026                   -215625.3   
                        2027                   -223209.7   
                        2028                   -231896.4   
                   URM  2025                   -365841.5   
                        2026                   -378594.0   
                        2027                   -391911.6   
                        2028                   -407160.7   
                   URS  2025                   -222939.4   
                        2026                   -228726.9   
                        2027                   -234048.1   
                        2028                   -240047.4   

                              Expected Change in GPG Per Capita ($)  \
Jurisdiction       ACLG Year                                          
Northern Territory RTL  2025                                   30.9   
                        2026                                   31.8   
                        2027                                   32.8   
                        2028                                   33.9   
                   RTM  2025                                    4.9   
                        2026                                    5.2   
                        2027                                    5.3   
                        2028                                    5.6   
                   RTS  2025                                   -8.7   
                        2026                                   -8.9   
                        2027                                   -9.1   
                        2028                                   -9.3   
                   RTX  2025                                   13.0   
                        2026                                   13.6   
                        2027                                   14.0   
                        2028                                   14.5   
                   UCC  2025                                   -8.7   
                        2026                                   -8.9   
                        2027                                   -9.1   
                        2028                                   -9.3   
                   UFS  2025                                   -8.7   
                        2026                                   -8.9   
                        2027                                   -9.1   
                        2028                                   -9.3   
                   URM  2025                                   -8.7   
                        2026                                   -8.9   
 

## A 10% Per Capita Minimum Grant

### New South Wales

In [13]:
display_data(sim_grants_10_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                   1117014.0   
                     2026                   1160472.3   
                     2027                   1203912.9   
                     2028                   1246003.5   
                RAM  2025                   1048626.0   
                     2026                   1087698.0   
                     2027                   1128473.4   
                     2028                   1168110.1   
                RAS  2025                   1451990.0   
                     2026                   1507168.1   
                     2027                   1559933.8   
                     2028                   1608535.5   
                RAV  2025                   1127224.4   
                     2026                   1171336.1   
                     2027                   1213872.3   
                     2028                   1258218.2   
                RTL  2025                   1656789.3   
                     2026                   1722652.9   
                     2027                   1776968.4   
                     2028                   1846534.8   
                RTM  2025                   1848690.4   
                     2026                   1923269.3   
                     2027                   1997306.2   
                     2028                   2068261.3   
                RTX  2025                     77203.1   
                     2026                     80359.1   
                     2027                     83297.3   
                     2028                     86249.8   
                UCC  2025                  -4165018.6   
                     2026                  -4329811.9   
                     2027                  -4497520.3   
                     2028                  -4669609.3   
                UDL  2025                  -1423822.9   
                     2026                  -1480162.1   
                     2027                  -1537492.3   
                     2028                  -1596323.1   
                UDM  2025                   -787569.9   
                     2026                   -818760.3   
                     2027                   -850554.6   
                     2028                   -883109.5   
                UDS  2025                   -228225.1   
                     2026                   -237375.9   
                     2027                   -246533.4   
                     2028                   -256091.5   
                UDV  2025                  -4435724.6   
                     2026                  -4611326.5   
                     2027                  -4791872.2   
                     2028                  -4976588.2   
                UFL  2025                  -1288728.9   
                     2026                  -1342057.7   
                     2027                  -1399601.8   
                     2028                  -1453556.0   
                UFM  2025                   -745806.5   
                     2026                   -774974.9   
                     2027                   -805981.9   
                     2028                   -837285.2   
                UFV  2025                  -2408606.9   
                     2026                  -2514175.2   
                     2027                  -2617970.9   
                     2028                  -2722895.9   
                URL  2025                    303581.5   
                     2026                    319557.1   
                     2027                    339170.4   
                     2028                    362276.4   
                URM  2025                    547640.8   
                     2026                    572250.3   
                     2027                    597668.9   
                     2028                    623835.6   
                URS  2025            

### Victoria

In [14]:
display_data(sim_grants_10_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                    849336.4   
                  2026                    885593.8   
                  2027                    922369.5   
                  2028                    961272.2   
             RAM  2025                    947798.9   
                  2026                    986627.8   
                  2027                   1031080.9   
                  2028                   1071388.3   
             RAV  2025                    813570.0   
                  2026                    849857.1   
                  2027                    886225.7   
                  2028                    924203.1   
             UCC  2025                  -3245055.3   
                  2026                  -3387625.3   
                  2027                  -3531572.6   
                  2028                  -3678954.0   
             UDL  2025                  -1677509.8   
                  2026                  -1752402.6   
                  2027                  -1826404.7   
                  2028                  -1904032.3   
             UDV  2025                  -2004372.3   
                  2026                  -2092665.7   
                  2027                  -2185865.6   
                  2028                  -2283856.3   
             UFL  2025                    586679.6   
                  2026                    589331.4   
                  2027                    608131.1   
                  2028                    625603.9   
             UFM  2025                    -12630.5   
                  2026                    -13089.0   
                  2027                    -13519.0   
                  2028                    -17011.2   
             UFS  2025                    623741.3   
                  2026                    653664.0   
                  2027                    687150.1   
                  2028                    722947.1   
             UFV  2025                   -271698.5   
                  2026                   -298265.8   
                  2027                   -320963.3   
                  2028                   -349337.3   
             URL  2025                   1676191.2   
                  2026                   1748081.2   
                  2027                   1834298.2   
                  2028                   1936918.0   
             URM  2025                   1372769.0   
                  2026                   1433398.8   
                  2027                   1497364.4   
                  2028                   1563575.9   
             URS  2025                   1148476.1   
                  2026                   1207625.7   
                  2027                   1262138.8   
                  2028                   1314181.4   
             URV  2025                    568028.3   
                  2026                    649044.9   
                  2027                    685654.2   
                  2028                    743189.8   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Victoria     RAL  2025                                  120.9     50462.0  
                  2026                                  125.0     50851.0  
                  2027                                  129.3     51217.0  
                  2028                                  133.7     51602.0  
             RAM  2025                                  242.0      3917.0  
                  2026                                  250.0      3947.0  
                  2027                                  259.3      3976.0  
                  2028                                  267.5      4005.0  
             RAV  2025                                   52.6    246987.0  
                  2026                                 

### Queensland

In [15]:
display_data(sim_grants_10_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                   2698854.3   
                  2026                   2812724.5   
                  2027                   2918825.6   
                  2028                   3015921.2   
             RAV  2025                   1665784.9   
                  2026                   1737592.9   
                  2027                   1807476.6   
                  2028                   1877392.0   
             RTL  2025                   2328512.7   
                  2026                   2424104.0   
                  2027                   2517765.3   
                  2028                   2614946.9   
             RTM  2025                   1706899.5   
                  2026                   1776220.8   
                  2027                   1846320.9   
                  2028                   1915749.6   
             RTS  2025                   1683089.9   
                  2026                   1750291.8   
                  2027                   1820164.1   
                  2028                   1890252.6   
             RTX  2025                   1671461.1   
                  2026                   1743928.2   
                  2027                   1811283.1   
                  2028                   1884403.4   
             UCC  2025                 -24039866.1   
                  2026                 -25073254.7   
                  2027                 -26094479.8   
                  2028                 -27138214.3   
             UFM  2025                   -706046.2   
                  2026                   -735580.2   
                  2027                   -764682.3   
                  2028                   -794290.1   
             UFS  2025                   -321713.1   
                  2026                   -336409.5   
                  2027                   -351131.3   
                  2028                   -365324.6   
             URL  2025                  -1977797.3   
                  2026                  -2054591.2   
                  2027                  -2131572.2   
                  2028                  -2209928.1   
             URM  2025                   -521575.1   
                  2026                   -540970.3   
                  2027                   -560323.5   
                  2028                   -580440.3   
             URS  2025                    509580.8   
                  2026                    529229.1   
                  2027                    547496.5   
                  2028                    569430.5   
             URV  2025                  -7299266.5   
                  2026                  -7594897.2   
                  2027                  -7889522.0   
                  2028                  -8189953.4   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Queensland   RAM  2025                                  600.8      4492.0  
                  2026                                  618.5      4548.0  
                  2027                                  634.4      4601.0  
                  2028                                  648.0      4654.0  
             RAV  2025                                  125.0    105395.0  
                  2026                                  128.7    106725.0  
                  2027                                  132.4    107956.0  
                  2028                                  135.9    109216.0  
             RTL  2025                                  440.4     47507.0  
                  2026                                  452.5     48105.0  
                  2027                                  465.1     48660.0  
                  2028                                  477.7     49227.0  
             RTM  2025                     

### South Australia

In [16]:
display_data(sim_grants_10_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                    425595.8   
                     2026                    440320.1   
                     2027                    455265.2   
                     2028                    471177.7   
                RAM  2025                    325628.8   
                     2026                    336560.1   
                     2027                    347956.5   
                     2028                    358775.3   
                RAS  2025                    268311.9   
                     2026                    277450.0   
                     2027                    286440.0   
                     2028                    295858.4   
                RAV  2025                    579765.3   
                     2026                    601505.6   
                     2027                    623859.3   
                     2028                    648465.3   
                RSG  2025                    -77902.2   
                     2026                    -80040.7   
                     2027                    -81916.9   
                     2028                    -83885.3   
                UCC  2025                   -501698.9   
                     2026                   -518895.8   
                     2027                   -536201.6   
                     2028                   -554145.0   
                UDL  2025                  -1949694.1   
                     2026                  -2016756.5   
                     2027                  -2084195.8   
                     2028                  -2154096.3   
                UDM  2025                   -965914.1   
                     2026                   -999018.7   
                     2027                  -1032356.5   
                     2028                  -1066885.7   
                UDS  2025                   -345425.7   
                     2026                   -357269.7   
                     2027                   -369191.7   
                     2028                   -381541.7   
                UDV  2025                  -1602019.7   
                     2026                  -1656379.2   
                     2027                  -1717085.6   
                     2028                  -1778957.5   
                UFL  2025                   1919342.5   
                     2026                   1969372.4   
                     2027                   2025184.5   
                     2028                   2067120.1   
                UFM  2025                   -752374.9   
                     2026                   -778156.9   
                     2027                   -804138.3   
                     2028                   -831030.6   
                UFS  2025                    102916.9   
                     2026                    108893.3   
                     2027                    113324.2   
                     2028                    118943.3   
                UFV  2025                  -1820360.0   
                     2026                  -1884175.5   
                     2027                  -1954215.9   
                     2028                  -2020097.9   
                URM  2025                   -470966.3   
                     2026                   -487710.4   
                     2027                   -504839.8   
                     2028                   -521017.3   
                URS  2025                    548230.2   
                     2026                    567872.9   
                     2027                    589143.5   
                     2028                    610074.2   

                           Expected Change in GPG Per Capita ($)  Population  
Jurisdiction    ACLG Year                                                     
South Australia RAL  2025                                   50.1     83192.0  
                     2026  

### Western Australia

In [17]:
display_data(sim_grants_10_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                    400197.1   
                       2026                    419256.4   
                       2027                    437879.9   
                       2028                    456509.4   
                  RAM  2025                    221075.5   
                       2026                    232308.1   
                       2027                    242405.0   
                       2028                    253324.9   
                  RAS  2025                    310015.0   
                       2026                    323847.7   
                       2027                    336736.5   
                       2028                    350021.8   
                  RAV  2025                    404831.4   
                       2026                    421206.5   
                       2027                    437391.6   
                       2028                    454602.8   
                  RSG  2025                   -182720.7   
                       2026                   -187231.5   
                       2027                   -191876.0   
                       2028                   -195777.4   
                  RTL  2025                    414342.0   
                       2026                    433512.0   
                       2027                    454005.9   
                       2028                    474180.7   
                  RTM  2025                    546559.6   
                       2026                    570206.5   
                       2027                    593874.9   
                       2028                    617635.5   
                  RTS  2025                    482469.2   
                       2026                    502788.5   
                       2027                    522876.2   
                       2028                    543868.4   
                  RTX  2025                    731767.2   
                       2026                    764854.6   
                       2027                    796306.7   
                       2028                    828480.1   
                  UCC  2025                   -598848.0   
                       2026                   -626302.1   
                       2027                   -652966.7   
                       2028                   -680132.7   
                  UDL  2025                  -2112586.2   
                       2026                  -2209473.7   
                       2027                  -2303518.9   
                       2028                  -2399348.0   
                  UDM  2025                   -953855.9   
                       2026                   -997595.9   
                       2027                  -1040061.8   
                       2028                  -1083325.9   
                  UDS  2025                   -444836.9   
                       2026                   -465245.6   
                       2027                   -485042.5   
                       2028                   -505217.5   
                  UDV  2025                  -3396727.3   
                       2026                  -3552350.1   
                       2027                  -3703533.9   
                       2028                  -3857509.9   
                  UFL  2025                  -1809358.5   
                       2026                  -1892331.1   
                       2027                  -1972873.5   
                       2028                  -2054941.4   
                  UFM  2025                   -848163.1   
                       2026                   -888418.8   
                       2027                   -927442.7   
                       2028                   -969077.6   
                  UFS  2025                   -160844.7   
                       2026                   -169608.5   
                 

### Tasmania

In [18]:
display_data(sim_grants_10_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                    301420.0   
                  2026                    309403.3   
                  2027                    318456.7   
                  2028                    328752.3   
             RAM  2025                    128259.0   
                  2026                    131746.4   
                  2027                    135383.5   
                  2028                    139471.8   
             RAS  2025                    256508.3   
                  2026                    263131.0   
                  2027                    271697.5   
                  2028                    279988.0   
             RAV  2025                    158836.6   
                  2026                    163487.3   
                  2027                    167251.5   
                  2028                    171889.5   
             UCC  2025                  -1183445.7   
                  2026                  -1217913.3   
                  2027                  -1253736.4   
                  2028                  -1294198.5   
             UFM  2025                  -1093892.3   
                  2026                  -1125688.9   
                  2027                  -1158701.1   
                  2028                  -1196007.3   
             UFS  2025                     88426.1   
                  2026                     92360.8   
                  2027                     93873.6   
                  2028                     96519.7   
             URM  2025                  -1002099.9   
                  2026                  -1029075.9   
                  2027                  -1055129.7   
                  2028                  -1085218.8   
             URS  2025                    230203.3   
                  2026                    236047.5   
                  2027                    242999.6   
                  2028                    250826.0   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Tasmania     RAL  2025                                   44.8     41480.0  
                  2026                                   45.9     41569.0  
                  2027                                   47.1     41646.0  
                  2028                                   48.5     41736.0  
             RAM  2025                                   27.3     23683.0  
                  2026                                   28.0     23735.0  
                  2027                                   28.7     23779.0  
                  2028                                   29.4     23831.0  
             RAS  2025                                  195.8      2672.0  
                  2026                                  200.4      2678.0  
                  2027                                  206.5      2683.0  
                  2028                                  212.3      2688.0  
             RAV  2025                                   12.0    105349.0  
                  2026                                   12.4    105628.0  
                  2027                                   12.6    105922.0  
                  2028                                   13.0    106243.0  
             UCC  2025                                  -17.4     68013.0  
                  2026                                  -17.8     68307.0  
                  2027                                  -18.2     68707.0  
                  2028                                  -18.7     69107.0  
             UFM  2025                                  -17.4    173305.0  
                  2026                                  -17.8    173977.0  
                  2027                                  -18.2    174856.0  
                  2028                                  -18.7    175746

### Northern Territory

In [19]:
display_data(sim_grants_10_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                    507604.7   
                        2026                    524184.4   
                        2027                    540993.0   
                        2028                    559994.8   
                   RTM  2025                     37523.9   
                        2026                     39115.4   
                        2027                     40105.2   
                        2028                     41979.8   
                   RTS  2025                     -8787.8   
                        2026                     -9018.2   
                        2027                     -9228.7   
                        2028                     -9468.2   
                   RTX  2025                      4775.1   
                        2026                      4889.6   
                        2027                      5081.4   
                        2028                      5264.5   
                   UCC  2025                  -1520999.9   
                        2026                  -1574045.8   
                        2027                  -1629408.0   
                        2028                  -1692814.0   
                   UFS  2025                   -416714.9   
                        2026                   -431250.6   
                        2027                   -446419.4   
                        2028                   -463792.8   
                   URM  2025                   -731683.0   
                        2026                   -757188.1   
                        2027                   -783823.2   
                        2028                   -814321.4   
                   URS  2025                   -445818.6   
                        2026                   -457395.8   
                        2027                   -467943.9   
                        2028                   -479990.7   

                              Expected Change in GPG Per Capita ($)  \
Jurisdiction       ACLG Year                                          
Northern Territory RTL  2025                                   61.7   
                        2026                                   63.6   
                        2027                                   65.6   
                        2028                                   67.8   
                   RTM  2025                                    9.7   
                        2026                                   10.1   
                        2027                                   10.4   
                        2028                                   10.9   
                   RTS  2025                                  -17.4   
                        2026                                  -17.8   
                        2027                                  -18.2   
                        2028                                  -18.6   
                   RTX  2025                                   26.4   
                        2026                                   27.0   
                        2027                                   28.1   
                        2028                                   28.9   
                   UCC  2025                                  -17.4   
                        2026                                  -17.8   
                        2027                                  -18.2   
                        2028                                  -18.6   
                   UFS  2025                                  -17.4   
                        2026                                  -17.8   
                        2027                                  -18.2   
                        2028                                  -18.6   
                   URM  2025                                  -17.4   
                        2026                                  -17.8   
 

## A World With No Per Capita Minimum Grant

### New South Wales

In [20]:
display_data(sim_grants_0_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                   1677369.4   
                     2026                   1741371.7   
                     2027                   1805340.8   
                     2028                   1871968.4   
                RAM  2025                   1573707.8   
                     2026                   1633834.4   
                     2027                   1694420.2   
                     2028                   1757304.0   
                RAS  2025                   2181406.9   
                     2026                   2257246.0   
                     2027                   2345499.1   
                     2028                   2421556.6   
                RAV  2025                   1690373.2   
                     2026                   1753361.4   
                     2027                   1818739.0   
                     2028                   1885656.1   
                RTL  2025                   2487279.6   
                     2026                   2584077.9   
                     2027                   2678056.0   
                     2028                   2779487.0   
                RTM  2025                   2772231.8   
                     2026                   2890068.1   
                     2027                   2996469.1   
                     2028                   3118078.5   
                RTX  2025                    115568.1   
                     2026                    120495.9   
                     2027                    124733.9   
                     2028                    129926.1   
                UCC  2025                  -6247527.8   
                     2026                  -6494717.9   
                     2027                  -6746280.4   
                     2028                  -7004414.0   
                UDL  2025                  -2135734.4   
                     2026                  -2220243.2   
                     2027                  -2306238.5   
                     2028                  -2394484.7   
                UDM  2025                  -1181479.0   
                     2026                  -1228241.4   
                     2027                  -1275793.8   
                     2028                  -1324770.9   
                UDS  2025                   -342272.6   
                     2026                   -355783.4   
                     2027                   -369666.0   
                     2028                   -383981.7   
                UDV  2025                  -6653969.7   
                     2026                  -6919612.0   
                     2027                  -7188115.3   
                     2028                  -7465037.4   
                UFL  2025                  -1924164.8   
                     2026                  -2000053.6   
                     2027                  -2082586.7   
                     2028                  -2167522.5   
                UFM  2025                  -1117281.8   
                     2026                  -1164315.0   
                     2027                  -1209504.5   
                     2028                  -1258427.7   
                UFV  2025                  -3608473.3   
                     2026                  -3752552.7   
                     2027                  -3909209.3   
                     2028                  -4063565.3   
                URL  2025                    454533.6   
                     2026                    483538.7   
                     2027                    509211.4   
                     2028                    534282.5   
                URM  2025                    818232.5   
                     2026                    854087.6   
                     2027                    893605.0   
                     2028                    934672.2   
                URS  2025            

### Victoria

In [21]:
display_data(sim_grants_0_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                   1272936.9   
                  2026                   1326685.1   
                  2027                   1381046.8   
                  2028                   1435604.2   
             RAM  2025                   1425898.7   
                  2026                   1477485.9   
                  2027                   1527938.7   
                  2028                   1588410.3   
             RAV  2025                   1219339.7   
                  2026                   1276437.3   
                  2027                   1330510.1   
                  2028                   1386405.7   
             UCC  2025                  -4867583.0   
                  2026                  -5081437.9   
                  2027                  -5297359.0   
                  2028                  -5518431.0   
             UDL  2025                  -2515305.2   
                  2026                  -2626175.1   
                  2027                  -2738035.4   
                  2028                  -2854404.5   
             UDV  2025                  -3016356.9   
                  2026                  -3165489.9   
                  2027                  -3302901.5   
                  2028                  -3449229.4   
             UFL  2025                    873816.6   
                  2026                    889937.1   
                  2027                    928522.9   
                  2028                    971831.4   
             UFM  2025                    -16596.7   
                  2026                    -19216.1   
                  2027                    -27319.1   
                  2028                    -26694.4   
             UFS  2025                    939411.1   
                  2026                    981466.8   
                  2027                   1023831.6   
                  2028                   1064494.8   
             UFV  2025                   -399138.0   
                  2026                   -422659.0   
                  2027                   -461135.5   
                  2028                   -485955.9   
             URL  2025                   2502727.9   
                  2026                   2625905.4   
                  2027                   2773694.7   
                  2028                   2884339.8   
             URM  2025                   2061357.2   
                  2026                   2154150.2   
                  2027                   2245367.0   
                  2028                   2348551.3   
             URS  2025                   1726912.9   
                  2026                   1802891.8   
                  2027                   1883675.4   
                  2028                   1970599.0   
             URV  2025                    882204.8   
                  2026                   1009948.6   
                  2027                   1120713.4   
                  2028                   1178406.3   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Victoria     RAL  2025                                  181.2     50462.0  
                  2026                                  187.4     50851.0  
                  2027                                  193.7     51217.0  
                  2028                                  199.8     51602.0  
             RAM  2025                                  364.0      3917.0  
                  2026                                  374.3      3947.0  
                  2027                                  384.3      3976.0  
                  2028                                  396.6      4005.0  
             RAV  2025                                   78.9    246987.0  
                  2026                                 

### Queensland

In [22]:
display_data(sim_grants_0_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                   4065437.4   
                  2026                   4237248.7   
                  2027                   4400268.3   
                  2028                   4567250.7   
             RAV  2025                   2502623.2   
                  2026                   2606075.6   
                  2027                   2707234.3   
                  2028                   2811204.0   
             RTL  2025                   3481612.8   
                  2026                   3614960.7   
                  2027                   3756827.9   
                  2028                   3899923.5   
             RTM  2025                   2561312.7   
                  2026                   2669377.9   
                  2027                   2774378.2   
                  2028                   2883403.9   
             RTS  2025                   2526331.9   
                  2026                   2628400.9   
                  2027                   2733112.7   
                  2028                   2840820.9   
             RTX  2025                   2505802.8   
                  2026                   2607318.4   
                  2027                   2707442.2   
                  2028                   2813537.6   
             UCC  2025                 -36059799.2   
                  2026                 -37609882.1   
                  2027                 -39141719.8   
                  2028                 -40707321.6   
             UFM  2025                  -1059632.6   
                  2026                  -1104159.6   
                  2027                  -1148593.5   
                  2028                  -1193450.8   
             UFS  2025                   -484229.7   
                  2026                   -505750.3   
                  2027                   -526157.2   
                  2028                   -549403.4   
             URL  2025                  -2966695.9   
                  2026                  -3081886.9   
                  2027                  -3197358.3   
                  2028                  -3314892.2   
             URM  2025                   -783060.0   
                  2026                   -812775.2   
                  2027                   -843428.7   
                  2028                   -873306.2   
             URS  2025                    761911.8   
                  2026                    799174.3   
                  2027                    834558.7   
                  2028                    862290.2   
             URV  2025                 -10948899.8   
                  2026                 -11392345.8   
                  2027                 -11834283.1   
                  2028                 -12284930.1   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Queensland   RAM  2025                                  905.0      4492.0  
                  2026                                  931.7      4548.0  
                  2027                                  956.4      4601.0  
                  2028                                  981.4      4654.0  
             RAV  2025                                  187.9    105395.0  
                  2026                                  193.2    106725.0  
                  2027                                  198.4    107956.0  
                  2028                                  203.7    109216.0  
             RTL  2025                                  659.8     47507.0  
                  2026                                  677.5     48105.0  
                  2027                                  696.2     48660.0  
                  2028                                  714.1     49227.0  
             RTM  2025                     

### South Australia

In [23]:
display_data(sim_grants_0_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                    636514.9   
                     2026                    657769.4   
                     2027                    679474.9   
                     2028                    703343.8   
                RAM  2025                    488901.2   
                     2026                    505230.4   
                     2027                    521621.0   
                     2028                    538059.4   
                RAS  2025                    402840.5   
                     2026                    416646.5   
                     2027                    429806.5   
                     2028                    442647.8   
                RAV  2025                    870557.1   
                     2026                    902989.0   
                     2027                    932797.0   
                     2028                    963481.9   
                RSG  2025                   -116821.6   
                     2026                   -120091.8   
                     2027                   -122814.0   
                     2028                   -125756.2   
                UCC  2025                   -752548.4   
                     2026                   -778343.7   
                     2027                   -804302.4   
                     2028                   -831217.4   
                UDL  2025                  -2924647.1   
                     2026                  -3025278.8   
                     2027                  -3126278.7   
                     2028                  -3231350.6   
                UDM  2025                  -1448871.2   
                     2026                  -1498528.1   
                     2027                  -1548534.8   
                     2028                  -1600328.5   
                UDS  2025                   -518138.6   
                     2026                   -535904.6   
                     2027                   -553787.6   
                     2028                   -572312.5   
                UDV  2025                  -2410213.6   
                     2026                  -2492118.5   
                     2027                  -2578899.9   
                     2028                  -2673890.3   
                UFL  2025                   2896254.3   
                     2026                   2980251.3   
                     2027                   3065528.0   
                     2028                   3191532.9   
                UFM  2025                  -1128562.3   
                     2026                  -1167235.4   
                     2027                  -1206207.5   
                     2028                  -1246545.9   
                UFS  2025                    153739.3   
                     2026                    162936.7   
                     2027                    172562.8   
                     2028                    183274.5   
                UFV  2025                  -2716978.7   
                     2026                  -2821270.8   
                     2027                  -2923389.5   
                     2028                  -3027974.7   
                URM  2025                   -705337.3   
                     2026                   -729542.9   
                     2027                   -753019.3   
                     2028                   -779040.9   
                URS  2025                    820368.9   
                     2026                    850339.6   
                     2027                    885612.8   
                     2028                    916035.1   

                           Expected Change in GPG Per Capita ($)  Population  
Jurisdiction    ACLG Year                                                     
South Australia RAL  2025                                   75.0     83192.0  
                     2026  

### Western Australia

In [24]:
display_data(sim_grants_0_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                    601797.9   
                       2026                    629659.2   
                       2027                    655760.2   
                       2028                    685167.4   
                  RAM  2025                    331757.6   
                       2026                    347952.5   
                       2027                    363432.7   
                       2028                    379752.4   
                  RAS  2025                    464672.4   
                       2026                    485075.3   
                       2027                    505030.8   
                       2028                    525411.7   
                  RAV  2025                    606822.8   
                       2026                    634778.6   
                       2027                    661112.0   
                       2028                    688320.5   
                  RSG  2025                   -274680.1   
                       2026                   -282279.3   
                       2027                   -289493.7   
                       2028                   -296535.9   
                  RTL  2025                    622329.5   
                       2026                    652779.6   
                       2027                    682694.9   
                       2028                    712616.2   
                  RTM  2025                    823177.4   
                       2026                    859778.3   
                       2027                    892704.0   
                       2028                    925918.7   
                  RTS  2025                    725068.4   
                       2026                    756265.5   
                       2027                    787605.4   
                       2028                    818364.5   
                  RTX  2025                   1096497.2   
                       2026                   1142223.2   
                       2027                   1188559.1   
                       2028                   1235123.4   
                  UCC  2025                   -898272.0   
                       2026                   -939453.2   
                       2027                   -979450.1   
                       2028                  -1020199.0   
                  UDL  2025                  -3168879.3   
                       2026                  -3314210.5   
                       2027                  -3455278.4   
                       2028                  -3599022.0   
                  UDM  2025                  -1430783.8   
                       2026                  -1496393.9   
                       2027                  -1560092.7   
                       2028                  -1624988.9   
                  UDS  2025                   -667255.3   
                       2026                   -697868.5   
                       2027                   -727563.7   
                       2028                   -757826.2   
                  UDV  2025                  -5094932.8   
                       2026                  -5328496.8   
                       2027                  -5556189.5   
                       2028                  -5787765.3   
                  UFL  2025                  -2714037.7   
                       2026                  -2838496.6   
                       2027                  -2959310.2   
                       2028                  -3082412.1   
                  UFM  2025                  -1272220.7   
                       2026                  -1328726.7   
                       2027                  -1386335.6   
                       2028                  -1444445.4   
                  UFS  2025                   -249392.2   
                       2026                   -260478.0   
                 

### Tasmania

In [25]:
display_data(sim_grants_0_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                    452212.9   
                  2026                    465033.4   
                  2027                    477408.0   
                  2028                    492256.1   
             RAM  2025                    193219.6   
                  2026                    199820.2   
                  2027                    205922.1   
                  2028                    212101.3   
             RAS  2025                    388054.1   
                  2026                    399320.1   
                  2027                    409791.2   
                  2028                    419892.9   
             RAV  2025                    238347.1   
                  2026                    245166.2   
                  2027                    251206.7   
                  2028                    259770.4   
             UCC  2025                  -1775168.5   
                  2026                  -1826870.0   
                  2027                  -1880604.5   
                  2028                  -1941297.8   
             UFM  2025                  -1640838.5   
                  2026                  -1688533.3   
                  2027                  -1738051.6   
                  2028                  -1794011.0   
             UFS  2025                    128430.1   
                  2026                    131209.6   
                  2027                    135009.4   
                  2028                    139949.2   
             URM  2025                  -1503149.8   
                  2026                  -1543613.8   
                  2027                  -1582694.6   
                  2028                  -1627828.2   
             URS  2025                    344355.2   
                  2026                    352254.0   
                  2027                    364534.3   
                  2028                    375259.4   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Tasmania     RAL  2025                                   67.2     41480.0  
                  2026                                   68.9     41569.0  
                  2027                                   70.6     41646.0  
                  2028                                   72.7     41736.0  
             RAM  2025                                   41.1     23683.0  
                  2026                                   42.4     23735.0  
                  2027                                   43.5     23779.0  
                  2028                                   44.7     23831.0  
             RAS  2025                                  296.4      2672.0  
                  2026                                  304.3      2678.0  
                  2027                                  311.6      2683.0  
                  2028                                  318.7      2688.0  
             RAV  2025                                   18.0    105349.0  
                  2026                                   18.5    105628.0  
                  2027                                   18.9    105922.0  
                  2028                                   19.5    106243.0  
             UCC  2025                                  -26.1     68013.0  
                  2026                                  -26.7     68307.0  
                  2027                                  -27.4     68707.0  
                  2028                                  -28.1     69107.0  
             UFM  2025                                  -26.1    173305.0  
                  2026                                  -26.7    173977.0  
                  2027                                  -27.4    174856.0  
                  2028                                  -28.1    175746

### Northern Territory

In [26]:
display_data(sim_grants_0_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                    760979.4   
                        2026                    785266.3   
                        2027                    810033.5   
                        2028                    838275.4   
                   RTM  2025                     56944.7   
                        2026                     59466.3   
                        2027                     62402.5   
                        2028                     66091.6   
                   RTS  2025                    -13181.7   
                        2026                    -13527.3   
                        2027                    -13843.1   
                        2028                    -14202.3   
                   RTX  2025                      7123.2   
                        2026                      7426.6   
                        2027                      7776.8   
                        2028                      8158.3   
                   UCC  2025                  -2281499.9   
                        2026                  -2361068.7   
                        2027                  -2444112.0   
                        2028                  -2539221.0   
                   UFS  2025                   -625072.4   
                        2026                   -646875.8   
                        2027                   -669629.1   
                        2028                   -695689.3   
                   URM  2025                  -1097524.5   
                        2026                  -1135782.1   
                        2027                  -1175734.9   
                        2028                  -1221482.0   
                   URS  2025                   -668726.9   
                        2026                   -686099.0   
                        2027                   -701880.0   
                        2028                   -719870.5   

                              Expected Change in GPG Per Capita ($)  \
Jurisdiction       ACLG Year                                          
Northern Territory RTL  2025                                   92.6   
                        2026                                   95.4   
                        2027                                   98.3   
                        2028                                  101.6   
                   RTM  2025                                   14.7   
                        2026                                   15.4   
                        2027                                   16.3   
                        2028                                   17.4   
                   RTS  2025                                  -26.1   
                        2026                                  -26.7   
                        2027                                  -27.3   
                        2028                                  -28.0   
                   RTX  2025                                   39.4   
                        2026                                   41.0   
                        2027                                   43.0   
                        2028                                   44.8   
                   UCC  2025                                  -26.1   
                        2026                                  -26.7   
                        2027                                  -27.3   
                        2028                                  -28.0   
                   UFS  2025                                  -26.1   
                        2026                                  -26.7   
                        2027                                  -27.3   
                        2028                                  -28.0   
                   URM  2025                                  -26.1   
                        2026                                  -26.7   
 

## And for sanity reasons, status quo

### New South Wales

In [27]:
display_data(sim_grants_30_melted, "New South Wales")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
New South Wales RAL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RTL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RTM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RTX  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UCC  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URS  2025            

### Victoria

In [28]:
display_data(sim_grants_30_melted, "Victoria")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Victoria     RAL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UCC  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UDL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UDV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Victoria     RAL  2025                                    0.0     50462.0  
                  2026                                    0.0     50851.0  
                  2027                                    0.0     51217.0  
                  2028                                    0.0     51602.0  
             RAM  2025                                    0.0      3917.0  
                  2026                                    0.0      3947.0  
                  2027                                    0.0      3976.0  
                  2028                                    0.0      4005.0  
             RAV  2025                                    0.0    246987.0  
                  2026                                 

### Queensland

In [29]:
display_data(sim_grants_30_melted, "Queensland")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Queensland   RAM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RTX  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UCC  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Queensland   RAM  2025                                    0.0      4492.0  
                  2026                                    0.0      4548.0  
                  2027                                    0.0      4601.0  
                  2028                                    0.0      4654.0  
             RAV  2025                                    0.0    105395.0  
                  2026                                    0.0    106725.0  
                  2027                                    0.0    107956.0  
                  2028                                    0.0    109216.0  
             RTL  2025                                    0.0     47507.0  
                  2026                                    0.0     48105.0  
                  2027                                    0.0     48660.0  
                  2028                                    0.0     49227.0  
             RTM  2025                     

### South Australia

In [30]:
display_data(sim_grants_30_melted, "South Australia")

Expected Change in GPG ($)  \
Jurisdiction    ACLG Year                               
South Australia RAL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RAV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                RSG  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UCC  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UDV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFL  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                UFV  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URM  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   
                URS  2025                         0.0   
                     2026                         0.0   
                     2027                         0.0   
                     2028                         0.0   

                           Expected Change in GPG Per Capita ($)  Population  
Jurisdiction    ACLG Year                                                     
South Australia RAL  2025                                    0.0     83192.0  
                     2026  

### Western Australia

In [31]:
display_data(sim_grants_30_melted, "Western Australia")

Expected Change in GPG ($)  \
Jurisdiction      ACLG Year                               
Western Australia RAL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RAM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RAS  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RAV  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RSG  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTS  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  RTX  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UCC  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDS  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UDV  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UFL  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UFM  2025                         0.0   
                       2026                         0.0   
                       2027                         0.0   
                       2028                         0.0   
                  UFS  2025                         0.0   
                       2026                         0.0   
                 

### Tasmania

In [32]:
display_data(sim_grants_30_melted, "Tasmania")

Expected Change in GPG ($)  \
Jurisdiction ACLG Year                               
Tasmania     RAL  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             RAV  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UCC  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             UFS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URM  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   
             URS  2025                         0.0   
                  2026                         0.0   
                  2027                         0.0   
                  2028                         0.0   

                        Expected Change in GPG Per Capita ($)  Population  
Jurisdiction ACLG Year                                                     
Tasmania     RAL  2025                                    0.0     41480.0  
                  2026                                    0.0     41569.0  
                  2027                                    0.0     41646.0  
                  2028                                    0.0     41736.0  
             RAM  2025                                    0.0     23683.0  
                  2026                                    0.0     23735.0  
                  2027                                    0.0     23779.0  
                  2028                                    0.0     23831.0  
             RAS  2025                                    0.0      2672.0  
                  2026                                    0.0      2678.0  
                  2027                                    0.0      2683.0  
                  2028                                    0.0      2688.0  
             RAV  2025                                    0.0    105349.0  
                  2026                                    0.0    105628.0  
                  2027                                    0.0    105922.0  
                  2028                                    0.0    106243.0  
             UCC  2025                                    0.0     68013.0  
                  2026                                    0.0     68307.0  
                  2027                                    0.0     68707.0  
                  2028                                    0.0     69107.0  
             UFM  2025                                    0.0    173305.0  
                  2026                                    0.0    173977.0  
                  2027                                    0.0    174856.0  
                  2028                                    0.0    175746

### Northern Territory

In [33]:
display_data(sim_grants_30_melted, "Northern Territory")

Expected Change in GPG ($)  \
Jurisdiction       ACLG Year                               
Northern Territory RTL  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   RTM  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   RTS  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   RTX  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   UCC  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   UFS  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   URM  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   
                   URS  2025                         0.0   
                        2026                         0.0   
                        2027                         0.0   
                        2028                         0.0   

                              Expected Change in GPG Per Capita ($)  \
Jurisdiction       ACLG Year                                          
Northern Territory RTL  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   RTM  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   RTS  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   RTX  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   UCC  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   UFS  2025                                    0.0   
                        2026                                    0.0   
                        2027                                    0.0   
                        2028                                    0.0   
                   URM  2025                                    0.0   
                        2026                                    0.0   
 